# Fire Bucket Distribution

We want to merge the fire risk scores with the fire incidents file, and see how many properties actually had an incident after. 

## Loading the Data

In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [2]:
fire_new = pd.read_csv("datasets/Fire_Incidents_New.csv", low_memory=False, encoding = 'utf-8',dtype={'street':'str','number':'str', 'CALL_CREATED_DATE': 'str'})
fire_inspections = pd.read_csv("datasets/Fire_Inspections.csv", low_memory=False, dtype={'parcel':'str'})
results = pd.read_csv("datasets/Results.csv", low_memory=False, dtype={'PROPERTYADDRESS':'str','PROPERTYHOUSENUM':'str'})
parcels = pd.read_csv("datasets/parcels.csv", low_memory=False, dtype={'PIN':'str'})
pittdata = pd.read_csv("datasets/pittdata.csv", low_memory=False, encoding = "ISO-8859-1", dtype={'PROPERTYADDRESS':'str','PROPERTYHOUSENUM':'str','CLASSDESC':'str'})

## Merging Parcels with Addresses in order to merge Fire Inspections data at Parcel Level

In [3]:
pitt_risk_parcels = pd.merge(left=pittdata,right=parcels, how='left', left_on='PARID', right_on='PIN')
parcel_address = pitt_risk_parcels[['PROPERTYHOUSENUM', 'PROPERTYADDRESS', 'PARID', 'MUNIDESC', 'CLASSDESC']]
parcel_address.head()

,PROPERTYHOUSENUM,PROPERTYADDRESS,PARID,MUNIDESC,CLASSDESC
0,564,FORBES AVE,0002K00173000000,1st Ward - PITTSBURGH,COMMERCIAL
1,952,VICKROY ST,0002K00190000000,1st Ward - PITTSBURGH,GOVERNMENT
2,1001,BLUFF ST,0002K00192000000,1st Ward - PITTSBURGH,GOVERNMENT
3,900,LOCUST ST,0002K00194000000,1st Ward - PITTSBURGH,GOVERNMENT
4,0,LOCUST ST,0002K00201000000,1st Ward - PITTSBURGH,GOVERNMENT


## Fire Incidents DataFrame Cleaning

In [4]:
fire_new['descript'] = fire_new['descript'].str.strip()
remove_descript = ['System malfunction, Other',
                   # 'Smoke detector activation, no fire - unintentional']
                   # 'Alarm system activation, no fire - unintentional']
                   'Detector activation, no fire - unintentional', 'Smoke detector activation due to malfunction',
                   'Dispatched & cancelled en route', 'Dispatched & cancelled on arrival',
                   'EMS call, excluding vehicle accident with injury', 'Medical assist, assist EMS crew',
                   'Emergency medical service, other', 'Good intent call, Other', 'Rescue, EMS incident, other',
                   'Medical Alarm Activation (No Medical Service Req)', 'Motor Vehicle Accident with no injuries',
                   'No Incident found on arrival at dispatch address', 'Unintentional transmission of alarm, Other',
                   'Motor vehicle accident with injuries', 'Vehicle accident, general cleanup', 'Power line down',
                   'Person in distress, Other', 'Cable/Telco Wires Down', 'Service Call, other',
                   'Vehicle Accident canceled en route', 'Lock-out', 'False alarm or false call, Other',
                   'Assist police or other governmental agency', 'Special type of incident, Other',
                   'Alarm system sounded due to malfunction', 'Motor vehicle/pedestrian accident (MV Ped)',
                   'Assist invalid ', 'Malicious, mischievous false call, Other', 'Accident, potential accident, Other',
                   'Assist invalid', 'EMS call, party transported by non-fire agency', 'Rescue or EMS standby',
                   'Public service assistance, Other', 'Police matter', 'Lock-in (if lock out , use 511 )',
                   'Sprinkler activation, no fire - unintentional', 'Wrong location',
                   'Local alarm system, malicious false alarm', 'Authorized controlled burning',
                   'Water problem, Other',
                   # 'Smoke or odor removal']
                   'Passenger vehicle fire', 'CO detector activation due to malfunction',
                   'Authorized controlled burning', 'Steam, vapor, fog or dust thought to be smoke', 'Overheated motor',
                   'Local alarm system, malicious false alarm', 'Central station, malicious false alarm',
                   'Public service',
                   # 'Building or structure weakened or collapsed'
                   'Heat detector activation due to malfunction', 'Citizen complaint',
                   'Municipal alarm system, malicious false alarm', 'Sprinkler activation due to malfunction',
                   'Severe weather or natural disaster, Other', 'Water evacuation', 'Breakdown of light ballast',
                   'Extrication of victim(s) from vehicle', 'Flood assessment', 'Telephone, malicious false alarm',
                   'Cover assignment, standby, moveup', 'Road freight or transport vehicle fire']

In [5]:
for descript in remove_descript:
    fire_new = fire_new[fire_new.descript != descript]
fire_new = fire_new[fire_new['full.code'].str.strip()  != '540 - Animal problem, Other']
fire_new = fire_new[fire_new['full.code'].str.strip()  != '5532 - Public Education (Station Visit)']
fire_new = fire_new[fire_new['full.code'].str.strip()  != '353 - Removal of victim(s) from stalled elevator']

In [6]:
#correcting problems with the street column
fire_new['street'] = fire_new['street'].replace(to_replace=', PGH', value='', regex=True)
fire_new['street'] = fire_new['street'].replace(to_replace=', P', value='', regex=True)
fire_new['street'] = fire_new['street'].replace(to_replace=',', value='', regex=True)
fire_new['street'] = fire_new['street'].replace(to_replace='#.*', value='', regex=True)
fire_new['street'] = fire_new['street'].str.strip()
fire_new['number'] = fire_new['number'].str.strip()

In [7]:
# drop duplicates
fire_new = fire_new.drop_duplicates()

In [19]:
#convert it to datetime and then check how many properties had a call
fire_new = fire_new[['CALL_CREATED_DATE', 'number', 'street', 'CALL_TYPE_FINAL', 'full.code']]
fire_new['CALL_CREATED_DATE'] = pd.to_datetime(fire_new['CALL_CREATED_DATE'])

#make full.code into different fire columns
fire_new['111 - Building Fire'] = np.where(fire_new['full.code'].astype(str).str[:3] == '111', '1', '0')
fire_new['Building Fire'] = np.where((fire_new['full.code'].astype(str).str[:2] == '11') & (fire_new['full.code'].astype(str).str[:3] != '112'), '1', '0')
fire_new['Code 100s'] = np.where(fire_new['full.code'].astype(str).str[0] == '1', '1', '0')
fire_new['Any Fire Code'] = '1'

Now, we want to merge it with address.

In [20]:
pitt_fire = pd.merge(left=parcel_address,right=fire_new, how='left', left_on =['PROPERTYADDRESS','PROPERTYHOUSENUM'],
        right_on = ['street','number'])

In [21]:
del pitt_fire['street']
del pitt_fire['number']

Now, I transform the full.code into different type of fire columns

## Fire Inspections Cleaning

In [22]:
fire_inspections['parcel'] = fire_inspections['parcel'].str.strip()
fire_inspections = fire_inspections[['parcel', 'reported', 'codeset', 'code', 'descript', 'completed']]
fire_inspections['reported'] = pd.to_datetime(fire_inspections['reported'])

In [23]:
pitt_inspect = pd.merge(left=parcel_address, right=fire_inspections, how='left', left_on=['PARID'], right_on=['parcel'])
del pitt_inspect['PARID']
pitt_inspect.head()

,PROPERTYHOUSENUM,PROPERTYADDRESS,MUNIDESC,CLASSDESC,parcel,reported,codeset,code,descript,completed
0,564,FORBES AVE,1st Ward - PITTSBURGH,COMMERCIAL,0002K00173000000,2018-01-18,VIOLATION,1003.6,Means of Egress Continuity ...,2018-05-15 00:00:00
1,564,FORBES AVE,1st Ward - PITTSBURGH,COMMERCIAL,0002K00173000000,2018-01-18,VIOLATION,1006.3,Illumination Emergency Power ...,2018-05-15 00:00:00
2,564,FORBES AVE,1st Ward - PITTSBURGH,COMMERCIAL,0002K00173000000,2018-01-18,VIOLATION,1011.5.3,Power Source - Exit Signs ...,2018-05-15 00:00:00
3,564,FORBES AVE,1st Ward - PITTSBURGH,COMMERCIAL,0002K00173000000,2018-01-18,VIOLATION,1022.8,Floor numbers in stairwell ...,2018-05-15 00:00:00
4,564,FORBES AVE,1st Ward - PITTSBURGH,COMMERCIAL,0002K00173000000,2018-01-18,VIOLATION,107.1,Maintenance of Safeguards ...,2018-05-15 00:00:00


## Filtering out Properties not in Pittsburgh

Getting rid of outliers

In [27]:
pitt_fire = pitt_fire[pitt_fire['MUNIDESC'].str.contains("Ward|Ingram|Wilkinsburg",na=False)]
pitt_inspect = pitt_inspect[pitt_inspect['MUNIDESC'].str.contains("Ward|Ingram|Wilkinsburg",na=False)]
# get rid of residential properteis in inspections
pitt_inspect = pitt_inspect[pitt_inspect.CLASSDESC!='RESIDENTIAL']

pitt_fire = pitt_fire[pitt_fire.PROPERTYHOUSENUM!= '0']
pitt_fire = pitt_fire[pitt_fire.PROPERTYHOUSENUM!= 'NaN']
pitt_fire = pitt_fire[pitt_fire.PROPERTYADDRESS!= '']
pitt_fire = pitt_fire[pitt_fire.PROPERTYADDRESS!= 'NaN']
pitt_fire.loc[pitt_fire['111 - Building Fire'] != '1', '111 - Building Fire'] = '0'
pitt_fire.loc[pitt_fire['Building Fire'] != '1', 'Building Fire'] = '0'
pitt_fire.loc[pitt_fire['Code 100s'] != '1', 'Code 100s'] = '0'
pitt_fire.loc[pitt_fire['Any Fire Code'] != '1', 'Any Fire Code'] = '0'

pitt_inspect = pitt_inspect[pitt_inspect.PROPERTYADDRESS!= '']
pitt_inspect = pitt_inspect[pitt_inspect.PROPERTYHOUSENUM!= '0']

## Making Commercial Fire Dataset

In [28]:
pitt_fire_c = pitt_fire[pitt_fire.CLASSDESC!='RESIDENTIAL']
pitt_fire_c = pitt_fire_c.groupby(['PROPERTYADDRESS', 'PROPERTYHOUSENUM'], as_index=False).max()

## Making Residential Fire Dataset

In [29]:
pitt_fire_r = pitt_fire[pitt_fire.CLASSDESC =='RESIDENTIAL']
pitt_fire_r.head()

,PROPERTYHOUSENUM,PROPERTYADDRESS,PARID,MUNIDESC,CLASSDESC,CALL_CREATED_DATE,CALL_TYPE_FINAL,full.code,111 - Building Fire,Building Fire,Code 100s,Any Fire Code
228,300,4TH AVE,0001H00327020100,1st Ward - PITTSBURGH,RESIDENTIAL,NaT,NaN,NaN,0.0,0.0,0.0,0.0
229,306,4TH AVE,0001H00327020200,1st Ward - PITTSBURGH,RESIDENTIAL,2015-08-26,False Alarm,"743 - Smoke detector activation, no fire - uni...",0.0,0.0,0.0,1.0
230,306,4TH AVE,0001H00327020200,1st Ward - PITTSBURGH,RESIDENTIAL,2015-10-09,False Alarm,"712 - Direct tie to FD, malicious false alarm ...",0.0,0.0,0.0,1.0
231,306,4TH AVE,0001H00327020200,1st Ward - PITTSBURGH,RESIDENTIAL,2016-05-25,False Alarm,"5001 - Smoke Detector Activation, No Fire ...",0.0,0.0,0.0,1.0
232,300,4TH AVE,0001H00327020300,1st Ward - PITTSBURGH,RESIDENTIAL,NaT,NaN,NaN,0.0,0.0,0.0,0.0


## Merge with Results of Commerical Model 

In [30]:
results = results[['PROPERTYADDRESS', 'PROPERTYHOUSENUM', 'RiskScore']]

In [31]:
pitt_fire_c = pd.merge(left=results,right=pitt_fire_c, how='left', left_on =['PROPERTYADDRESS','PROPERTYHOUSENUM'],
        right_on = ['PROPERTYADDRESS','PROPERTYHOUSENUM'])

In [32]:
pitt_inspect =  pd.merge(left=results, right=pitt_inspect, how='right', left_on=['PROPERTYADDRESS','PROPERTYHOUSENUM'],
        right_on = ['PROPERTYADDRESS','PROPERTYHOUSENUM'])
pitt_fire_c.head()

,PROPERTYADDRESS,PROPERTYHOUSENUM,RiskScore,PARID,MUNIDESC,CLASSDESC,CALL_CREATED_DATE,CALL_TYPE_FINAL,full.code,111 - Building Fire,Building Fire,Code 100s,Any Fire Code
0,ALGER ST,1,0.001652,0054K00230000000,15th Ward - PITTSBURGH,GOVERNMENT,2017-06-12,Hazardous Conditions,412 - Gas leak (natural gas or LPG) ...,0.0,0.0,0.0,1.0
1,ALLEGHENY AVE,1,0.002512,0007M00500000000,21st Ward - PITTSBURGH,GOVERNMENT,2018-07-12,Service Call,"745 - Alarm system activation, no fire - unint...",1.0,1.0,1.0,1.0
2,BIGELOW SQ,1,0.000172,0002B00096000000,2nd Ward - PITTSBURGH,COMMERCIAL,2018-07-03,Service Call,"745 - Alarm system activation, no fire - unint...",0.0,0.0,1.0,1.0
3,HEINZ ST,1,0.000197,0024P00120000000,23rd Ward - PITTSBURGH,OTHER,2017-03-27,Good Intent,"745 - Alarm system activation, no fire - unint...",0.0,0.0,0.0,1.0
4,TRIMONT LN,1,0.006536,0006M00312D16600,19th Ward - PITTSBURGH,COMMERCIAL,2017-07-08,False Alarm,"745 - Alarm system activation, no fire - unint...",0.0,0.0,0.0,1.0


## Merge with Results of Residential Model

In [33]:
## get the residential model csv and we can do this part very easily

## Commerical Fire Distribution

In [34]:
cutoff = datetime.datetime.now() - relativedelta(months=6)
cutoffdate = cutoff.strftime("%m/%d/%Y")
#data_over = pitt_fire_c[pitt_fire_c.CALL_CREATED_DATE > cutoffdate]

high = pitt_fire_c[pitt_fire_c['RiskScore'] >= 0.70]
medium = pitt_fire_c[(pitt_fire_c['RiskScore'] < 0.70) & (pitt_fire_c['RiskScore'] >= 0.40)]
low = pitt_fire_c[(pitt_fire_c['RiskScore'] >= 0) & (pitt_fire_c['RiskScore'] < 0.40)]

In [35]:
high.drop_duplicates(subset=['PROPERTYADDRESS', 'PROPERTYHOUSENUM'])

,PROPERTYADDRESS,PROPERTYHOUSENUM,RiskScore,PARID,MUNIDESC,CLASSDESC,CALL_CREATED_DATE,CALL_TYPE_FINAL,full.code,111 - Building Fire,Building Fire,Code 100s,Any Fire Code
10,5TH AVE,1001,0.961461,0307G00179000000,6th Ward - McKEESPORT,GOVERNMENT,2018-07-09,Good Intent,"745 - Alarm system activation, no fire - unint...",0.0,0.0,1.0,1.0
21,BENTON AVE,1028,0.829405,0114H00250000000,27th Ward - PITTSBURGH,OTHER,2018-06-01,Service Call,"745 - Alarm system activation, no fire - unint...",0.0,1.0,1.0,1.0
30,KNOX AVE,107,0.791552,0014R00121000000,30th Ward - PITTSBURGH,COMMERCIAL,2018-09-06,Service Call,"746 - Carbon monoxide detector activation, no ...",1.0,1.0,1.0,1.0
59,TECUMSEH ST,111,0.974320,0056K00255000000,15th Ward - PITTSBURGH,GOVERNMENT,2018-04-06,Service Call,"745 - Alarm system activation, no fire - unint...",1.0,1.0,1.0,1.0
158,5TH AVE,1601,0.931283,0002H00088000000,3rd Ward - PITTSBURGH,OTHER,2018-01-08,Fire,"745 - Alarm system activation, no fire - unint...",1.0,1.0,1.0,1.0
165,LINCOLN AVE,1621,0.940763,0172P00240000000,12th Ward - PITTSBURGH,GOVERNMENT,2018-04-13,Service Call,"745 - Alarm system activation, no fire - unint...",1.0,1.0,1.0,1.0
341,RHINE ST,2170,0.955103,0047F00195000000,26th Ward - PITTSBURGH,COMMERCIAL,2018-06-22,Good Intent,"745 - Alarm system activation, no fire - unint...",0.0,0.0,1.0,1.0
371,NOBLESTOWN RD,2350,0.906157,0039L00175000000,28th Ward - PITTSBURGH,COMMERCIAL,2017-03-13,Service Call,"651 - Smoke scare, odor of smoke ...",1.0,1.0,1.0,1.0
581,ATWOOD ST,326,0.979793,0028G00167000000,4th Ward - PITTSBURGH,COMMERCIAL,2017-06-28,Fire,"150 - Outside rubbish fire, Other ...",0.0,1.0,1.0,1.0
611,FORBES AVE,3333,0.701918,0028J00097000300,4th Ward - PITTSBURGH,COMMERCIAL,2018-07-13,Service Call,"745 - Alarm system activation, no fire - unint...",0.0,0.0,1.0,1.0


In [46]:
building_sum_h = high['111 - Building Fire'].sum()
building_fire_h = high['Building Fire'].sum()
code_100_h = high['Code 100s'].sum()
fire_h = high['Any Fire Code'].sum()

371.0

In [48]:
building_sum_m = medium['111 - Building Fire'].sum()
building_fire_m = medium['Building Fire'].sum()
code_100_m = medium['Code 100s'].sum()
fire_m = medium['Any Fire Code'].sum()

In [47]:
building_sum_l = low['111 - Building Fire'].sum()
building_fire_l = low['Building Fire'].sum()
code_100_l = low['Code 100s'].sum()
fire_l = low['Any Fire Code'].sum()

## Out of those that had a high risk score, how many of them actually had inspection violations? 

In [ ]:
pitt_inspect['viol_cutoff'] = np.where(pitt_inspect['RiskScore']>0.70, 1, 0)
pitt_inspect['risk_high'] = np.where((pitt_inspect['RiskScore'] >=0.70) & (pitt_inspect['reported'] > cutoff), 1, 0)
pitt_inspect.drop_duplicates(subset=['PROPERTYADDRESS', 'PROPERTYHOUSENUM'])

In [ ]:
has_viol_set = has_viol.drop_duplicates(subset=['PROPERTYADDRESS', 'PROPERTYHOUSENUM'])
high_bracket = has_viol_set[has_viol_set['RiskScore']> 0.70].sort_values(['RiskScore'], ascending=False)
high_bracket

In [ ]:
fire_new = pd.read_csv("datasets/Fire_Incidents_New.csv", low_memory=False, encoding = 'utf-8',dtype={'street':'str','number':'str', 'CALL_CREATED_DATE': 'str'})

In [ ]:
set(fire_new['full.code'])